# RAG Evaluation Dataset Generation

## Overview

This notebook is **Part 1 of 2** in the RAGAS Evaluation Demo. It demonstrates how to generate RAG (Retrieval-Augmented Generation) evaluation datasets using the SDG Hub framework. It creates question-answer pairs with ground truth context that can be used to evaluate RAG systems.

**Next:** After completing this notebook, proceed to **[`2.ragas-evaluation.ipynb`](2.ragas-evaluation.ipynb)** to evaluate your RAG system using RAGAS metrics.

## What This Notebook Does

This notebook will:

1. **Construct Input Dataset**: Show how to prepare documents with outlines for the RAG evaluation flow
2. **Generate RAG Evaluation Dataset**: Run the RAG Evaluation flow to create question-answer pairs with:
   - Topic extraction from documents
   - Conceptual question generation
   - Question evolution for better quality
   - Answer generation with grounding
   - Groundedness scoring and filtering
   - Ground truth context extraction
3. **Post-process for Evaluation**: Convert the output to evaluation-ready formats (e.g., for RAGAS)

In [ ]:
%pip install -r requirements.txt

In [ ]:
from datasets import Dataset
import pandas as pd
import json
import os

from sdg_hub import Flow, FlowRegistry

# Required to run the flow with async mode
import nest_asyncio

nest_asyncio.apply()

## Step 1: Prepare Input Dataset

The RAG Evaluation flow requires:
- **document**: The full text content of the document
- **document_outline**: A concise title or summary that represents the document

You can prepare this from various sources:
- PDF documents (extract text and create outlines)
- Text files
- Existing datasets
- Web content

Below are example functions to help construct the input dataset.

In [ ]:
def prepare_dataset_from_text(text: str, document_outline: str, chunk_size: int = 3000, overlap: int = 500):
    """
    Prepare dataset from a single text document by chunking it.
    
    Args:
        text: Full document text
        document_outline: Title or summary of the document
        chunk_size: Maximum characters per chunk
        overlap: Overlap between chunks to maintain context (must be < chunk_size)
        
    Returns:
        Dataset with document and document_outline columns
    """
    # Validate parameters
    if overlap >= chunk_size:
        raise ValueError(f"overlap ({overlap}) must be less than chunk_size ({chunk_size})")
    
    if chunk_size <= 0:
        raise ValueError(f"chunk_size must be positive, got {chunk_size}")
    
    # Simple chunking by character count with overlap
    chunks = []
    step_size = chunk_size - overlap
    
    for i in range(0, len(text), step_size):
        chunk = text[i:i + chunk_size]
        if chunk.strip():
            chunks.append(chunk)
    
    # Create dataset
    dataset = Dataset.from_dict({
        "document": chunks,
        "document_outline": [document_outline] * len(chunks)
    })
    
    print(f"Created {len(chunks)} chunks from document")
    return dataset


def prepare_dataset_from_pdf(pdf_path: str, document_outline: str, max_pages: int = None):
    """
    Prepare dataset from a PDF file.
    
    Args:
        pdf_path: Path to PDF file
        document_outline: Title or summary of the document
        max_pages: Maximum number of pages to process (None for all)
        
    Returns:
        Dataset with document and document_outline columns
    """
    try:
        from PyPDF2 import PdfReader
    except ImportError:
        raise ImportError("PyPDF2 is required. Install with: pip install PyPDF2")
    
    reader = PdfReader(pdf_path)
    text = ""
    
    pages_to_read = reader.pages[:max_pages] if max_pages else reader.pages
    for page in pages_to_read:
        text += page.extract_text() + "\n"
    
    return prepare_dataset_from_text(text, document_outline)

### Example: Create Dataset from IBM Annual Report

Here's an example using the IBM 2024 Annual Report. It will extract text from the first 20 pages and create chunks for processing.

In [ ]:
pdf_path = "ibm-annual-report-2024.pdf"

if not os.path.exists(pdf_path):
    raise FileNotFoundError(
        f"PDF file not found: {pdf_path}\n"
    )

input_dataset = prepare_dataset_from_pdf(pdf_path, "IBM 2024 Annual Report Summary", max_pages=20)
print(f"\nInput dataset columns: {input_dataset.column_names}")
print(f"Number of samples: {len(input_dataset)}")

## Step 2: Discover and Load the RAG Evaluation Flow

In [ ]:
# Get the RAG Evaluation flow
flow_name = "RAG Evaluation Dataset Flow"
flow_path = FlowRegistry.get_flow_path(flow_name)

flow = Flow.from_yaml(flow_path)

In [ ]:
def set_model_config(flow_object):
    """Configure the model for the flow based on environment variables."""
    model = os.getenv("INFERENCE_MODEL", "vllm-inference/qwen3-14b-awq")
    api_base = os.getenv("URL", "http://lsd-ragas-example-service:8321/v1")
    api_key = os.getenv("API_KEY", "fake")
    
    if model and not model.startswith("openai/") and not model.startswith("ollama/"):
        model = "openai/" + model
    
    print(f"Configuring model: {model}")
    
    flow_object.set_model_config(
        model=model,
        api_base=api_base if api_base else None,
        api_key=api_key if api_key else None,
    )
    
    return flow_object

# Configure the model
flow = set_model_config(flow)

## Step 4: Generate RAG Evaluation Dataset

Run the flow to generate question-answer pairs with ground truth context. The flow will:
1. Extract topics from documents
2. Generate conceptual questions
3. Evolve questions for better quality
4. Generate answers with grounding
5. Score groundedness and filter low-quality pairs
6. Extract ground truth context

In [ ]:
# Get runtime parameters
max_concurrency = int(os.getenv("MAX_CONCURRENCY", "10"))

# Optional: Configure runtime parameters for specific blocks
runtime_params = {}

print("This may take several minutes depending on dataset size and model speed...\n")

# Generate the dataset
generated_data = flow.generate(
    input_dataset, 
    runtime_params=runtime_params, 
    max_concurrency=max_concurrency
)

## Step 5: Post-process for Evaluation

Convert the generated dataset to evaluation-ready formats. This prepares the data for use with evaluation frameworks like RAGAS.


In [ ]:
from pathlib import Path

df = generated_data.to_pandas()

def prepare_for_ragas_evaluation(generated_df: pd.DataFrame, output_file: str = None):
    """
    Convert generated dataset to RAGAS evaluation format.
    
    RAGAS expects:
    - question: The question
    - answer: The generated answer
    - contexts: List of context strings (usually one)
    - ground_truth: The ground truth answer (can be same as answer or use ground_truth_context)
    
    Args:
        generated_df: DataFrame from flow generation
        output_file: Optional path to save JSONL file
        
    Returns:
        List of dictionaries in RAGAS format
    """
    ragas_data = []
    
    for _, row in generated_df.iterrows():
        question = row.get('question', '')
        answer = row.get('response', '')
        context = row.get('document', row.get('context', ''))
        ground_truth = row.get('ground_truth_context', answer)
        
        ragas_record = {
            "question": str(question),
            "answer": str(answer),
            "contexts": [str(context)] if context else [""],
            "ground_truth": str(ground_truth)
        }
        
        ragas_data.append(ragas_record)
    
    if output_file:
        output_file = Path(output_file)
        output_file.parent.mkdir(parents=True, exist_ok=True)

        with output_file.open("w") as f:
            for record in ragas_data:
                f.write(json.dumps(record, ensure_ascii=False) + "\n")

    return ragas_data

ragas_data = prepare_for_ragas_evaluation(df, output_file="rag_evaluation_dataset.jsonl")

# Save the full generated dataset
output_csv = "rag_evaluation_full_results.csv"
generated_data.to_csv(output_csv, index=False)
print(f"Saved full results to {output_csv}")

print(f"\n✅ Prepared {len(ragas_data)} records for evaluation")

## Summary

🎉 You have successfully:

1. ✅ Prepared input dataset with documents and outlines
2. ✅ Generated RAG evaluation dataset with question-answer pairs
3. ✅ Post-processed data for evaluation frameworks

### Generated Files

- `rag_evaluation_dataset.jsonl` - RAGAS-ready evaluation dataset
- `rag_evaluation_full_results.csv` - Full generated results with all metadata

### Next Steps

**➡️ Continue to the RAGAS Evaluation Notebook**

Now that you have generated your evaluation dataset, proceed to **[`2.ragas-evaluation.ipynb`](2.ragas-evaluation.ipynb)** to:
- Load the generated dataset
- Run RAGAS evaluation metrics using the Llama Stack RAGAS provider
- Analyze evaluation results and metrics

The second notebook will use the `rag_evaluation_dataset.jsonl` file created in this notebook to perform RAG evaluation.
